In [ ]:
import plot
import classify
import importlib
import preprocess
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# reload when updating code
importlib.reload(preprocess)
# mouse and date
mouse = 'NN28'
date = '230216'
# create folders to save files
paths = preprocess.create_folders(mouse, date)
# import data for mouse and date as dict
session_data = preprocess.load_data(paths)
# process and plot behavior
behavior = preprocess.process_behavior(session_data, paths)
# save masks so can run in matlab to process other planes
preprocess.cell_masks(paths, 0)

In [ ]:
# reload when updating code
importlib.reload(preprocess)
# grab activity 
deconvolved = preprocess.process_activity(paths, 'spks', 3, 0)
# normalize activity
norm_deconvolved = preprocess.normalize_deconvolved(deconvolved, behavior, paths, 0)
# gassuain filter acitivity
norm_moving_deconvolved_filtered = preprocess.difference_gaussian_filter(norm_deconvolved, 4, behavior, paths, 0)
# make trial averaged traces and basline subtract
mean_cs_1_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_1')
mean_cs_2_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_2')
# get sig cells
[cs_1_poscells, cs_1_negcells] = preprocess.sig_test(norm_deconvolved, behavior, 'cs_1')
[cs_2_poscells, cs_2_negcells] = preprocess.sig_test(norm_deconvolved, behavior, 'cs_2')
[both_poscells, both_sigcells] = preprocess.combine_sig(cs_1_poscells, cs_1_negcells, cs_2_poscells, cs_2_negcells)
# get idx of top cell differences
idx = preprocess.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells, cs_2_poscells, both_poscells, both_sigcells, paths, 1)

In [ ]:
# reload when updating code
importlib.reload(classify)
importlib.reload(preprocess)
# get prior for synchronous cue activity
prior = classify.prior(norm_moving_deconvolved_filtered, idx['cs_1'], idx['cs_2'], behavior, [])
# logistic regression
y_pred = classify.log_regression(behavior, norm_deconvolved, norm_moving_deconvolved_filtered, both_poscells, prior)
# process classified output
y_pred = classify.process_classified(y_pred, prior, paths, 1)
# sig reactivated cells
sig_cells = preprocess.sig_reactivated_cells(norm_deconvolved, norm_moving_deconvolved_filtered, idx, y_pred, behavior, paths, 1)

In [ ]:
# reload when updating code
importlib.reload(plot)
# plot heatmap of top cells
plot.sorted_map(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, idx['cs_1'], idx['cs_2'], 150, paths)
# plot mean reactivation probability after cues
plot.reactivation_rate(y_pred, behavior, paths, [])
# plot reactivation bias over time
plot.reactivation_bias(y_pred, behavior, paths, [], [])
# plot physical evoked reactivations
plot.reactivation_physical(y_pred, behavior, paths, [], [])
# plot activity change with reactivation rates over time
plot.activity_across_trials(norm_deconvolved, behavior, y_pred, idx, paths, [], [])
# plot activity control
plot.activity_control(norm_deconvolved, behavior, paths, [], [])
# plot reactivation raster
plot.reactivation_raster(behavior, norm_deconvolved, y_pred, idx['cs_1_df'], idx['cs_2_df'], idx['both'], paths, [])
#plot.sample_reactivation_raster(behavior, norm_deconvolved, y_pred, idx['cs_1_df'], idx['cs_2_df'], idx['both'], paths, 46500, 47500)

In [ ]:
S_overlap = cs_1_poscells + cs_2_poscells
S_overlap[S_overlap!=2] = 0
S1_only = both_poscells - cs_2_poscells
S2_only = both_poscells - cs_1_poscells

S1_only = preprocess.sort_cells(behavior, mean_cs_1_responses_df, S1_only, 'Mean', 2, 'descending').index
S2_only = preprocess.sort_cells(behavior, mean_cs_2_responses_df, S2_only, 'Mean', 2, 'descending').index
S_overlap = preprocess.sort_cells(behavior, mean_cs_2_responses_df+mean_cs_1_responses_df, S_overlap, 'Mean', 2,
                              'descending').index

print(len(S1_only))
print(len(S2_only))
print(len(S_overlap))
importlib.reload(plot)
plot.sorted_map(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, S1_only, S1_only, len(S1_only), paths)
#plot.sorted_map(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, S2_only, S2_only, len(S2_only), paths)
#plot.sorted_map(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, S_overlap, S_overlap, len(S_overlap), paths)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
# create folders to save files
paths = preprocess.create_folders(mouse, date)
# import data for mouse and date as dict
session_data = preprocess.load_data(paths)
# process and plot behavior
behavior = preprocess.process_behavior(session_data, paths)
# normalize activity
norm_deconvolved = preprocess.normalize_deconvolved([], behavior, paths, 0)
# Gaussian filter activity
norm_moving_deconvolved_filtered = preprocess.difference_gaussian_filter(norm_deconvolved, 4, behavior, paths, 0)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
import random
# random subset of cells
rand_cells = random.sample(range(0, len(norm_deconvolved)), int(len(norm_deconvolved)/10))
norm_deconvolved_subset = norm_deconvolved.loc[rand_cells, :]
norm_moving_deconvolved_filtered_subset = norm_moving_deconvolved_filtered[rand_cells, :]
# make trial averaged traces and baseline subtract
mean_cs_1_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved_subset, behavior, 'cs_1')
mean_cs_2_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved_subset, behavior, 'cs_2')
# get sig cells
[cs_1_poscells, cs_1_negcells] = preprocess.sig_test(norm_deconvolved_subset, behavior, 'cs_1')
[cs_2_poscells, cs_2_negcells] = preprocess.sig_test(norm_deconvolved_subset, behavior, 'cs_2')
[both_poscells, both_sigcells] = preprocess.combine_sig(cs_1_poscells, cs_1_negcells, cs_2_poscells,
                                                        cs_2_negcells)
# get idx of top cell differences
idx = preprocess.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells,
                           cs_2_poscells, both_poscells, both_sigcells, paths, 2)
# get idx of top cell differences
idx_original = preprocess.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells, cs_2_poscells, both_poscells, both_sigcells, paths, 0)
# get prior for synchronous cue activity
prior = classify.prior(norm_moving_deconvolved_filtered_subset, idx['cs_1'], idx['cs_2'], behavior, [])
# logistic regression
y_pred = classify.log_regression(behavior, norm_deconvolved_subset, norm_moving_deconvolved_filtered_subset,
                                 both_poscells, prior)
# process classified output
y_pred = classify.process_classified(y_pred, prior, paths, 2)
y_pred_original = classify.process_classified([], [], paths, 0)

import pandas as pd
import numpy as np
import plot_subset
importlib.reload(plot_subset)
plot_subset.sample_reactivation_raster(behavior, pd.DataFrame(np.array(norm_deconvolved_subset)), norm_deconvolved, y_pred, y_pred_original, 
                                       idx['cs_1_df'], idx['cs_2_df'], idx_original['cs_1_df'], idx_original['cs_2_df'], paths, 24500, 25500)

In [ ]:
# plot noise vs grating
base_path = 'D:/2p_data/scan/'
paths = 'D:/2p_data/scan/' + 'NN1' + '/' + '200310' + '_' + 'NN1'
from scipy.io import loadmat
import numpy as np
import pandas as pd
session_data = loadmat(paths + '/behavior_file.mat')
plane_path = paths + '/suite2p_plane_1/suite2p/plane0/'
accepted_cells = np.load(plane_path + 'iscell.npy')[:, 0]
all_activity = np.load(plane_path + 'F.npy')
all_activity = all_activity - np.load(plane_path + 'Fneu.npy')
activity = all_activity[accepted_cells == 1, :]
norm_vec = np.empty(len(activity))
norm_vec[:] = np.nan
for i in range(0, len(activity)):
    temp_deconvolved = activity[i, :][activity[i,:] > 0]
    temp_deconvolved = np.flip(np.sort(temp_deconvolved))
    norm_value = np.mean(temp_deconvolved[0:int(len(temp_deconvolved) / 100)])
    norm_vec[i] = norm_value
activity = pd.DataFrame(activity)
norm_deconvolved = activity.divide(norm_vec, axis=0)
norm_deconvolved = norm_deconvolved.to_numpy()

In [ ]:
index_frames = []
num_trials = 0
trials_idx = np.where(np.isin(session_data['condition'], 1, 2))[0]
trial_times = session_data['onsets'][trials_idx]
for i in trial_times:
    if i > 0:
        num_trials = num_trials + 1
        for j in range(-31, 31*4):
            index_frames.append(int(i) + j)
activity_task_idx = norm_deconvolved[:, index_frames]
activity_task_idx = np.reshape(activity_task_idx, (activity_task_idx.shape[0], num_trials, 31*5))
activity_task_mean = pd.DataFrame(activity_task_idx.mean(axis=1))
frames_before = 31
activity_task_mean_df_grating = activity_task_mean.subtract(activity_task_mean.iloc[:, int(frames_before / 2):frames_before].mean(axis=1), axis=0)
index_frames = []
num_trials = 0
trials_idx = np.where(np.isin(session_data['condition'], 3, 4))[0]
trial_times = session_data['onsets'][trials_idx]
for i in trial_times:
    if i > 0:
        num_trials = num_trials + 1
        for j in range(-31, 31*4):
            index_frames.append(int(i) + j)
activity_task_idx = norm_deconvolved[:, index_frames]
activity_task_idx = np.reshape(activity_task_idx, (activity_task_idx.shape[0], num_trials, 31*5))
activity_task_mean = pd.DataFrame(activity_task_idx.mean(axis=1))
frames_before = 31
activity_task_mean_df_noise = activity_task_mean.subtract(activity_task_mean.iloc[:, int(frames_before / 2):frames_before].mean(axis=1), axis=0)

In [ ]:
frames_before = 31
idx = []
mean_responses_idx = pd.DataFrame({'mean_fluorescence': activity_task_mean_df_grating.iloc[:, frames_before:frames_before*2].mean(axis=1)})
idx_grating = mean_responses_idx.sort_values(by=['mean_fluorescence'], ascending=0)
idx = []
mean_responses_idx = pd.DataFrame({'mean_fluorescence': activity_task_mean_df_noise.iloc[:, frames_before:frames_before*2].mean(axis=1)})
idx_noise = mean_responses_idx.sort_values(by=['mean_fluorescence'], ascending=0)


In [ ]:
sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid': False})
sns.set_style("ticks")
plt.figure(figsize=(5, 10))
plt.subplot(1, 2, 2)
sns.heatmap(activity_task_mean_df_grating.loc[idx_grating.index, :], vmin=0, vmax=.2, cmap = 'Greys', cbar=0)
plt.axvline(x=frames_before + .25, color='k', linestyle='-', linewidth=2, snap=False)
plt.axvline(x=frames_before * 2 + .25, color='k', linestyle='-', linewidth=2, snap=False)
plt.xticks([])
plt.yticks([])
plt.xlim(0, 31*4)
plt.subplot(1, 2, 1)
sns.heatmap(activity_task_mean_df_noise.loc[idx_noise.index, :], vmin=0, vmax=.2, cmap = 'Greys', cbar=0)
plt.axvline(x=frames_before + .25, color='k', linestyle='-', linewidth=2, snap=False)
plt.axvline(x=frames_before * 2 + .25, color='k', linestyle='-', linewidth=2, snap=False)
plt.xticks([])
plt.yticks([])
plt.xlim(0, 31*4)
plt.savefig(paths + 'cue_heatmap.png', bbox_inches='tight', dpi=500)

In [ ]:
from scipy import stats
index_frames = []
num_trials = 0
trials_idx = np.where(np.isin(session_data['condition'], 1, 2))[0]
trial_times = session_data['onsets'][trials_idx]
for i in trial_times:
    if i > 0:
        num_trials = num_trials + 1
        for j in range(-frames_before, frames_before):
            index_frames.append(int(i) + j)
activity_idx = activity[:, index_frames]
activity_idx = np.reshape(activity_idx, (activity_idx.shape[0], num_trials, frames_before * 2))
pos_sig_cells_grating = np.zeros(len(activity_idx))
for j in range(len(activity_idx)):
    before = np.reshape(activity_idx[j, :, 0:frames_before], (num_trials * frames_before))
    after = np.reshape(activity_idx[j, :, frames_before:frames_before * 2], (num_trials * frames_before))
    res = stats.ranksums(before, after)
    if res.statistic < 0 and res.pvalue < .01:
        pos_sig_cells_grating[j] = 1
index_frames = []
num_trials = 0
trials_idx = np.where(np.isin(session_data['condition'], 3, 4))[0]
trial_times = session_data['onsets'][trials_idx]
for i in trial_times:
    if i > 0:
        num_trials = num_trials + 1
        for j in range(-frames_before, frames_before):
            index_frames.append(int(i) + j)
activity_idx = activity[:, index_frames]
activity_idx = np.reshape(activity_idx, (activity_idx.shape[0], num_trials, frames_before * 2))
pos_sig_cells_noise = np.zeros(len(activity_idx))
for j in range(len(activity_idx)):
    before = np.reshape(activity_idx[j, :, 0:frames_before], (num_trials * frames_before))
    after = np.reshape(activity_idx[j, :, frames_before:frames_before * 2], (num_trials * frames_before))
    res = stats.ranksums(before, after)
    if res.statistic < 0 and res.pvalue < .01:
        pos_sig_cells_noise[j] = 1

In [ ]:
sum(pos_sig_cells_noise)
sum(pos_sig_cells_grating)

In [ ]:
# plot z motion
base_path = 'D:/2p_data/scan/'
paths = 'D:/2p_data/scan/' + 'NN28' + '/' + '230217' + '_' + 'NN28'
from scipy.io import loadmat
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d
plane_path = paths + '/suite2p_plane_1/suite2p/plane0/'
ops = np.load(plane_path + 'ops.npy', allow_pickle=True).item()
y_off = ops['yoff']
x_off = ops['xoff']
zcorr = ops['zcorr']
zcorr = np.argmax(gaussian_filter1d(zcorr.T.copy(), 2, axis=1), axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid': False})
sns.set_style("ticks")
plt.figure(figsize=(9, 6))
plt.subplot(3, 1, 1)
plt.plot(np.abs(x_off) * 1500/512)
plt.xticks([])
plt.ylabel('x motion\n(μm, abs)')
plt.xlim(4750, 5350)
plt.subplot(3, 1, 2)
plt.plot(np.abs(y_off) * 1500/512, c='k')
plt.xticks([])
plt.ylabel('y motion\n(μm, abs)')
plt.xlim(4750, 5350)
plt.subplot(3, 1, 3)
plt.plot(zcorr * 10, c='r')
plt.xlim(4750, 5350)
plt.xticks([4750, 4950, 5150, 5350], ['0', '20', '40', '60'])
plt.ylabel('z motion\n(μm, abs)')
plt.xlabel('Time (seconds)')
sns.despine()
plt.savefig(paths + '_zmotion.pdf', bbox_inches='tight', dpi=150)

In [ ]:
np.corrcoef(x_off[4000:4600], y_off[4000:4600])